In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.io as pio
pio.renderers.default = "notebook_connected"
import warnings
warnings.filterwarnings("ignore")

from pyecharts.charts import Bar, Grid, Line, Liquid, Page, Pie, Timeline
from pyecharts import options as opts


pd.options.display.float_format = '{:.2f}'.format
col = px.colors.qualitative.Pastel

In [2]:
act = pd.read_csv('2_act_info.csv')
cus = pd.read_csv('2_cus_info.csv')
iem = pd.read_csv('2_iem_info.csv')
trd_kr =  pd.read_csv('2_trd_kr.csv')
trd_oss =  pd.read_csv('2_trd_oss.csv')
cus = cus.astype({'tco_cus_grd_cd' : 'str'})
trd_kr['orr_dt'] = pd.to_datetime(trd_kr['orr_dt'].astype(str), format='%Y%m%d')
trd_oss['orr_dt'] = pd.to_datetime(trd_oss['orr_dt'].astype(str), format='%Y%m%d')

In [3]:
#연령대 카테고리 추가

cus.loc[(cus['cus_age'] >= 0) & (cus['cus_age'] < 10), 'age_cat'] = '10대 미만'
cus.loc[(cus['cus_age'] >= 10) & (cus['cus_age'] < 20), 'age_cat'] = '10대'
cus.loc[(cus['cus_age'] >= 20) & (cus['cus_age'] < 30), 'age_cat'] = '20대'
cus.loc[(cus['cus_age'] >= 30) & (cus['cus_age'] < 40), 'age_cat'] = '30대'
cus.loc[(cus['cus_age'] >= 40) & (cus['cus_age'] < 50),'age_cat'] = '40대'
cus.loc[(cus['cus_age'] >= 50) & (cus['cus_age'] < 60), 'age_cat'] = '50대'
cus.loc[(cus['cus_age'] >= 60) & (cus['cus_age'] < 70), 'age_cat'] = '60대'
cus.loc[cus['cus_age'] == 70, 'age_cat'] = '70대 이상'

#고객 등급, 투자성향 변수 변경

cus['tco_cus_grd_cd'] = cus['tco_cus_grd_cd'].replace({'01':'01 탑클래스','02':'02 골드','03':'03 로얄','04':'04 그린',
                                                       '05':'05 블루','09':'09 등급없음','_ ':'09 등급없음'})
cus['ivs_icn_cd'] = cus['ivs_icn_cd'].replace({'01':'01 안정형','02':'02 안정추구형','03':'03 위험중립형','04':'04 적극투자형',
                                               '05':'05 공격투자형','09':'09 전문투자자형','00':'정보 제공 미동의',
                                               '_ ':'등급없음', '_':'등급없음','-':'등급없음'})
cus = cus.rename({"sex_dit_cd":"sex", "cus_age":"age", "zip_ctp_cd": "home",
                  "tco_cus_grd_cd":"grade", "ivs_icn_cd":"ivst"},axis='columns')

In [4]:
actcus = pd.merge(act,cus,on='cus_id')
act_cus = actcus[['act_id','cus_id','act_opn_ym']]
act_cus.to_csv('act_cus.csv', index=False) #고객정보와 계좌정보 연결한 데이터

In [5]:
#개인당 국내계좌 개수 추출

count = {}
lists = list(actcus['cus_id'])

for i in lists:
    try: count[i] += 1
    except: count[i]=1
        
a = pd.DataFrame({'cus_id' : list(count.keys()), 
                  'num_act' : list(count.values())})

In [6]:
cus_info = pd.merge(cus, a, on='cus_id') #고객정보 데이터(10000명)

In [7]:
cus_info1 = pd.merge(cus_info, act, on='cus_id', how='left')
cus_info1

,cus_id,sex,age,home,grade,ivst,age_cat,num_act,act_id,act_opn_ym
0,1eec7e43dd6ea34d6be7a9484459e4db1a2da44c5bf311...,2,20,29,04 그린,등급없음,20대,1,fe689fbddf2bb32c1489d66f0ba0ef5da9478b82e7f56c...,200110
1,7e50c268acce884b1a5b78c9b5645b517c4639413c6dc4...,2,0,41,09 등급없음,등급없음,10대 미만,1,28e8c3cd0299c7436f44eb9f140960e969907f144d3c0d...,200604
2,36d3cd293c4b02a00103fa1ba33ffbee176173a55df75b...,1,70,27,03 로얄,등급없음,70대 이상,10,acb3a3e48fc80ebe89447b2bf29e79df801ef4881bad92...,199511
3,36d3cd293c4b02a00103fa1ba33ffbee176173a55df75b...,1,70,27,03 로얄,등급없음,70대 이상,10,8d5b316001d9c7d04c36ff37554111fa12196943d4e78b...,200702
4,36d3cd293c4b02a00103fa1ba33ffbee176173a55df75b...,1,70,27,03 로얄,등급없음,70대 이상,10,3ef2a4cfe5420010436ad7fe9513ef39631621c4092454...,199408
...,...,...,...,...,...,...,...,...,...,...
23954,347190a59df83bbf2d87888915b95cd5b5c52fb781a45e...,2,35,11,09 등급없음,등급없음,30대,4,530858638ae97f3eae6ba84318790ec8abbced1c3b9bc4...,200209
23955,347190a59df83bbf2d87888915b95cd5b5c52fb781a45e...,2,35,11,09 등급없음,등급없음,30대,4,6e76f7efd3e36a7b1d679417a14c413455a88bafdebc3c...,201807
23956,3ae218361d32511eec17e92b139f048ec2613ff6d56ac9...,1,35,31,05 블루,등급없음,30대,2,21f135109922a4393eaa622c5e40be9f75ef5073e7606a...,201310
23957,3ae218361d32511eec17e92b139f048ec2613ff6d56ac9...,1,35,31,05 블루,등급없음,30대,2,de5d6391de5e44d106fbbcd71343f071705bb92c4da923...,201410


In [8]:
cus_info1['act_opn_ym'] = cus_info1['act_opn_ym'].astype('str')
cus_info1['opn_ym'] = cus_info1['act_opn_ym'].str[:4]
cus_info1['opn_ym'] = cus_info1['opn_ym'].astype('int')

In [9]:
cus_info1

,cus_id,sex,age,home,grade,ivst,age_cat,num_act,act_id,act_opn_ym,opn_ym
0,1eec7e43dd6ea34d6be7a9484459e4db1a2da44c5bf311...,2,20,29,04 그린,등급없음,20대,1,fe689fbddf2bb32c1489d66f0ba0ef5da9478b82e7f56c...,200110,2001
1,7e50c268acce884b1a5b78c9b5645b517c4639413c6dc4...,2,0,41,09 등급없음,등급없음,10대 미만,1,28e8c3cd0299c7436f44eb9f140960e969907f144d3c0d...,200604,2006
2,36d3cd293c4b02a00103fa1ba33ffbee176173a55df75b...,1,70,27,03 로얄,등급없음,70대 이상,10,acb3a3e48fc80ebe89447b2bf29e79df801ef4881bad92...,199511,1995
3,36d3cd293c4b02a00103fa1ba33ffbee176173a55df75b...,1,70,27,03 로얄,등급없음,70대 이상,10,8d5b316001d9c7d04c36ff37554111fa12196943d4e78b...,200702,2007
4,36d3cd293c4b02a00103fa1ba33ffbee176173a55df75b...,1,70,27,03 로얄,등급없음,70대 이상,10,3ef2a4cfe5420010436ad7fe9513ef39631621c4092454...,199408,1994
...,...,...,...,...,...,...,...,...,...,...,...
23954,347190a59df83bbf2d87888915b95cd5b5c52fb781a45e...,2,35,11,09 등급없음,등급없음,30대,4,530858638ae97f3eae6ba84318790ec8abbced1c3b9bc4...,200209,2002
23955,347190a59df83bbf2d87888915b95cd5b5c52fb781a45e...,2,35,11,09 등급없음,등급없음,30대,4,6e76f7efd3e36a7b1d679417a14c413455a88bafdebc3c...,201807,2018
23956,3ae218361d32511eec17e92b139f048ec2613ff6d56ac9...,1,35,31,05 블루,등급없음,30대,2,21f135109922a4393eaa622c5e40be9f75ef5073e7606a...,201310,2013
23957,3ae218361d32511eec17e92b139f048ec2613ff6d56ac9...,1,35,31,05 블루,등급없음,30대,2,de5d6391de5e44d106fbbcd71343f071705bb92c4da923...,201410,2014


In [10]:
cond = cus_info1['opn_ym'] < 2018
cus_info1['opn_y'] = np.where(cond, '2018년_이전', cus_info1['opn_ym'])

In [11]:
df = cus_info1.groupby(['opn_y', 'age_cat']).count().reset_index()

In [12]:
df = df[['opn_y', 'age_cat', 'cus_id']]
df

,opn_y,age_cat,cus_id
0,2018,10대 미만,27
1,2018,20대,271
2,2018,30대,599
3,2018,40대,667
4,2018,50대,391
5,2018,60대,101
6,2018,70대 이상,34
7,2018년_이전,10대 미만,96
8,2018년_이전,20대,294
9,2018년_이전,30대,1476


In [13]:
df1 = df.pivot(index='age_cat',columns='opn_y',values='cus_id').reset_index()
df1 = df1.rename(columns={'2018': "year_2018", '2019': "year_2019", '2020': "year_2020"})

In [14]:
df1

opn_y,age_cat,year_2018,2018년_이전,year_2019,year_2020
0,10대 미만,27,96,28,113
1,20대,271,294,493,1007
2,30대,599,1476,719,1364
3,40대,667,3828,720,1342
4,50대,391,4488,451,805
5,60대,101,2837,141,161
6,70대 이상,34,1438,25,43


In [15]:
trace1 = go.Bar(x=df1.age_cat, y=df1.year_2018, name='2018년',marker_color=col[2])
trace2 = go.Bar(x=df1.age_cat, y=df1.year_2019, name='2019년',marker_color=col[1])
trace3 = go.Bar(x=df1.age_cat, y=df1.year_2020, name='2020년',marker_color=col[0])

fig = go.Figure(data=[trace1, trace2, trace3], layout=go.Layout(title='연도별 계좌 개설 개수'))
pyo.iplot(fig)

In [16]:
#2020년 10월까지만 데이터 존재
cus_info1[cus_info1.opn_ym == 2020].act_opn_ym.unique()

array(['202003', '202006', '202004', '202010', '202005', '202008',
       '202009', '202002', '202007', '202001'], dtype=object)

In [17]:
#일까지 정보가 없어서 에러가 나는 듯,,,
#cus_info1['act_opn_ym'] = cus_info1['act_opn_ym'].astype('str')
#pd.datetime(cus_info1['act_opn_ym'], format='%Y%m')

In [18]:
cus_info1['act_opn_ym'].unique()

array(['200110', '200604', '199511', '200702', '199408', '200002',
       '200004', '200112', '200606', '199812', '200103', '200802',
       '199910', '200003', '199904', '200612', '200006', '201204',
       '200108', '199908', '200506', '200810', '199909', '199307',
       '199603', '198903', '199101', '199208', '200111', '197603',
       '200512', '199811', '200402', '200405', '202003', '198909',
       '200008', '198801', '199306', '199707', '200608', '199302',
       '198711', '199701', '201112', '200707', '200510', '200711',
       '200306', '199102', '199509', '200805', '199802', '199110',
       '199809', '200703', '200704', '198702', '198811', '202006',
       '199402', '199803', '200507', '200705', '201707', '200511',
       '200803', '200504', '200602', '199907', '199201', '200011',
       '200212', '200210', '200409', '199801', '199906', '200501',
       '199808', '199409', '200503', '200407', '199806', '200804',
       '200205', '200607', '201012', '201405', '199503', '2007

In [41]:
cus_info1['act_opn_ym'] = cus_info1['act_opn_ym'].astype('int')
df_1920 = cus_info1[cus_info1['act_opn_ym'] > 201900]
df_20 = df_1920[(df_1920['age_cat'] == '20대') | (df_1920['age_cat'] == '30대')]
df2 = df_20.groupby(['act_opn_ym', 'age_cat']).count().reset_index()[['act_opn_ym', 'age_cat', 'cus_id']]

In [42]:
df3 = df2.pivot(index='act_opn_ym',columns='age_cat',values='cus_id').reset_index()
df3['act_opn'] = df3.act_opn_ym.astype('str').str[:4] + '-' + df3.act_opn_ym.astype('str').str[4:] 

In [43]:
df3

age_cat,act_opn_ym,20대,30대,act_opn
0,201901,42,55,2019-01
1,201902,32,61,2019-02
2,201903,49,38,2019-03
3,201904,38,54,2019-04
4,201905,62,102,2019-05
5,201906,12,43,2019-06
6,201907,47,43,2019-07
7,201908,32,74,2019-08
8,201909,36,51,2019-09
9,201910,36,71,2019-10


In [44]:
trace3 = go.Bar(x=df3.act_opn, y=df3['20대'], name='20대',marker_color=col[5])
trace4 = go.Bar(x=df3.act_opn, y=df3['30대'], name='30대',marker_color=col[6])


data = [trace3, trace4]

layout = go.Layout(title='월별 계좌개설 개수(2019년-2020년)', barmode='stack')

fig = go.Figure(data=data, layout=layout)
pyo.iplot(fig)

In [45]:
trace3 = go.Bar(x=df3.act_opn, y=df3['20대'], name='20대',marker_color='#F3BE50')
trace4 = go.Bar(x=df3.act_opn, y=df3['30대'], name='30대',marker_color='#2C66B5')


data = [trace3, trace4]

layout = go.Layout(title='월별 계좌개설 개수(2019년-2020년)')

fig = go.Figure(data=data, layout=layout)
fig.update_layout(font=dict(size=15,))
pyo.iplot(fig)

In [24]:
df_181920 = cus_info1[cus_info1['act_opn_ym'] > 201800]
df_2030 = df_181920[(df_181920['age_cat'] == '20대') | (df_181920['age_cat'] == '30대')]
df4 = df_2030.groupby(['act_opn_ym', 'age_cat']).count().reset_index()[['act_opn_ym', 'age_cat', 'cus_id']]

In [25]:
df4 = df4.pivot(index='act_opn_ym',columns='age_cat',values='cus_id').reset_index()
df4['act_opn'] = df4.act_opn_ym.astype('str').str[:4] + '-' + df4.act_opn_ym.astype('str').str[4:] 

In [26]:
trace3 = go.Bar(x=df4.act_opn, y=df4['20대'], name='20대',marker_color=col[5])
trace4 = go.Bar(x=df4.act_opn, y=df4['30대'], name='30대',marker_color=col[6])


data = [trace3, trace4]

layout = go.Layout(title='월별 계좌개설 개수(2018년-2020년)', barmode='stack')

fig = go.Figure(data=data, layout=layout)
pyo.iplot(fig)

In [27]:
trace3 = go.Bar(x=df4.act_opn, y=df4['20대'], name='20대',marker_color=col[4])
trace4 = go.Bar(x=df4.act_opn, y=df4['30대'], name='30대',marker_color=col[2])


data = [trace3, trace4]

layout = go.Layout(title='월별 계좌개설 개수(2018년-2020년)')

fig = go.Figure(data=data, layout=layout)
pyo.iplot(fig)

In [33]:
act = pd.read_csv('2_act_info.csv')
cus = pd.read_csv('2_cus_info.csv')
iem = pd.read_csv('2_iem_info.csv')
trd_kr =  pd.read_csv('2_trd_kr.csv')
trd_oss =  pd.read_csv('2_trd_oss.csv')
cus = cus.astype({'tco_cus_grd_cd' : 'str'})
trd_kr['orr_dt'] = pd.to_datetime(trd_kr['orr_dt'].astype(str), format='%Y%m%d')
trd_oss['orr_dt'] = pd.to_datetime(trd_oss['orr_dt'].astype(str), format='%Y%m%d')

trd_kr = pd.merge(act_cus, trd_kr, on='act_id') #거래-고객 정보 데이터
trd_kr['amount'] = trd_kr['cns_qty']*trd_kr['orr_pr'] #거래별 금액 계산
kr = trd_kr.rename({'orr_rtn_hur':'ord','lst_cns_hur':'real','sby_dit_cd':'buysell', 
                        'orr_mdi_dit_cd':'store', 'cns_qty':'qty', 'orr_pr':'price'},axis='columns')


In [34]:
cus_info = pd.read_csv('cus_info.csv')

In [35]:
df = kr
df['haha'] = 1
df_date = df[['orr_dt','haha']]
df_date = df_date.groupby(['orr_dt']).sum()
dff = df_date.resample('W').mean().reset_index()

df = pd.merge(df, cus_info, on='cus_id')

df2 = df.loc[df['age_cat'] == '20대']
df_date = df2[['orr_dt','haha']]
df_date = df_date.groupby(['orr_dt']).sum()
df_20 = df_date.resample('W').mean().reset_index()

df3 = df.loc[df['age_cat'] == '30대']
df_date = df3[['orr_dt','haha']]
df_date = df_date.groupby(['orr_dt']).sum()
df_30 = df_date.resample('W').mean().reset_index()

df4 = df.loc[(df['age_cat'] != '20대') & (df['age_cat'] != '30대')]
df_date = df4[['orr_dt','haha']]
df_date = df_date.groupby(['orr_dt']).sum()
df_2030 = df_date.resample('W').mean().reset_index()

In [32]:
df

,act_id,orr_dt,orr_ord,ord,real,iem_cd,buysell,qty,price,store,haha
0,6caf4de40251d37cbba310f0c6dd0ed8f4c7ca4ccfa675...,2019-01-02,1,12,13,A215600,2,35.00,74200.00,3,1
1,6caf4de40251d37cbba310f0c6dd0ed8f4c7ca4ccfa675...,2019-01-02,2,12,12,A001060,2,20.00,41100.00,3,1
2,effb34815e1b7efb34beb550f604ede8ac71a5a71edb30...,2019-01-02,1,14,14,A281820,2,20.00,9460.00,4,1
3,fea6c038cb7236a02d1e779262698a36219b4c9b527ab1...,2019-01-02,1,12,12,A006650,2,1.00,148500.00,3,1
4,fea6c038cb7236a02d1e779262698a36219b4c9b527ab1...,2019-01-02,2,12,13,A051360,2,10.00,6550.00,3,1
...,...,...,...,...,...,...,...,...,...,...,...
3312659,a96172625957ac18f6debba0c3283f1aa4608c1d19a404...,2020-06-30,2,13,13,A900250,1,1.00,1760.00,3,1
3312660,a96172625957ac18f6debba0c3283f1aa4608c1d19a404...,2020-06-30,3,16,16,A011930,2,2.00,1340.00,3,1
3312661,c8d469612160df7251dd5d434ca25b0b8e2f40ace9a37f...,2020-06-30,1,11,11,A019175,2,35.00,44000.00,3,1
3312662,1a046249ad088f30dae8038eb95d51427f10228677d4fb...,2020-06-30,1,16,16,A037950,1,400.00,1810.00,4,1


In [46]:
df2 = df.loc[(df['age_cat'] == '20대')&(df['covid'] == 'after')]
df_date = df2[['orr_dt','haha']]
df_date = df_date.groupby(['orr_dt']).sum()
df_20 = df_date.resample('W').mean().reset_index()

df3 = df.loc[(df['age_cat'] == '30대')&(df['covid'] == 'after')]
df_date = df3[['orr_dt','haha']]
df_date = df_date.groupby(['orr_dt']).sum()
df_30 = df_date.resample('W').mean().reset_index()


trace3 = go.Bar(x=df_20.orr_dt, y=df_20.haha, name='20대',marker_color='#F3BE50')
trace4 = go.Bar(x=df_30.orr_dt, y=df_30.haha, name='30대',marker_color='#2C66B5')

data = [trace3, trace4]

layout = go.Layout(title='30대 거래건수 흐름')

fig = go.Figure(data=data, layout=layout)
fig.update_layout(font=dict(size=15,))
pyo.iplot(fig)

In [37]:
data = [trace4]

layout = go.Layout(title='30대 거래건수 흐름', barmode='stack')

fig = go.Figure(data=data, layout=layout)
fig.update_layout(font=dict(size=15,))
pyo.iplot(fig)

In [49]:
cus_info = pd.read_csv('cus_info2.csv')

In [119]:
#고객별 종목 산업 개수 시각화

df2 = cus_info.loc[(cus_info['age_cat']=='20대') & (cus_info['sex'] == 1)].groupby(by=['covid']).median().reindex(['before','after']).reset_index()
df3 = cus_info.loc[(cus_info['age_cat']=='20대') & (cus_info['sex'] == 2)].groupby(by=['covid']).median().reindex(['before','after']).reset_index()
df4 = cus_info.loc[(cus_info['age_cat']=='30대') & (cus_info['sex'] == 1)].groupby(by=['covid']).median().reindex(['before','after']).reset_index()
df5 = cus_info.loc[(cus_info['age_cat']=='30대') & (cus_info['sex'] == 2)].groupby(by=['covid']).median().reindex(['before','after']).reset_index()

trace3 = go.Bar(x=['코로나 이전','코로나 이후'], y=df2.ind_cat, name='20대 남성',marker_color='#2C66B5')
trace4 = go.Bar(x=['코로나 이전','코로나 이후'], y=df3.ind_cat, name='20대 여성',marker_color='#F3BE50')
trace5 = go.Bar(x=['코로나 이전','코로나 이후'], y=df4.ind_cat, name='30대 남성',marker_color='#2C66B5')
trace6 = go.Bar(x=['코로나 이전','코로나 이후'], y=df5.ind_cat, name='30대 여성',marker_color='#F3BE50')

fig = go.Figure(data=[trace3, trace4, trace5, trace6], layout=go.Layout(title='고객별 산업 개수'))
fig.update_layout(font=dict(size=18,))
pyo.iplot(fig)

In [120]:
df2 = cus_info.loc[(cus_info['age_cat']=='20대') & (cus_info['sex'] == 1)].groupby(by=['covid']).median().reindex(['before','after']).reset_index()
df3 = cus_info.loc[(cus_info['age_cat']=='20대') & (cus_info['sex'] == 2)].groupby(by=['covid']).median().reindex(['before','after']).reset_index()
df4 = cus_info.loc[(cus_info['age_cat']=='30대') & (cus_info['sex'] == 1)].groupby(by=['covid']).median().reindex(['before','after']).reset_index()
df5 = cus_info.loc[(cus_info['age_cat']=='30대') & (cus_info['sex'] == 2)].groupby(by=['covid']).median().reindex(['before','after']).reset_index()

trace3 = go.Bar(x=['코로나 이전','코로나 이후'], y=df2.iem_num_kr, name='20대 남성',marker_color='#2C66B5')
trace4 = go.Bar(x=['코로나 이전','코로나 이후'], y=df3.iem_num_kr, name='20대 여성',marker_color='#F3BE50')
trace5 = go.Bar(x=['코로나 이전','코로나 이후'], y=df4.iem_num_kr, name='30대 남성',marker_color='#2C66B5')
trace6 = go.Bar(x=['코로나 이전','코로나 이후'], y=df5.iem_num_kr, name='30대 여성',marker_color='#F3BE50')

fig = go.Figure(data=[trace3, trace4, trace5, trace6], layout=go.Layout(title='고객별 상품 개수'))
fig.update_layout(font=dict(size=18,))
pyo.iplot(fig)

In [55]:
cus_info.loc[cus_info['freq'] == 0]

,cus_id,age_cat,sex,grade,class,risk,ivst,home,num_act,iem_num_kr,...,기타,산업재,소재,에너지,필수소비재,통신서비스,ind_cat,big_firm,small_firm,covid
3,001ee721662751620282570981b2b4e2c1d42a8ef9329e...,50대,1,02 골드,상,등급없음,등급없음,47,2,18.00,...,0.00,207.29,-168.74,0.00,0.00,45.74,6.00,-5.17,105.17,before
8,003a6123b8701df16de38ef7fc3b1fb67773cedcf7c913...,30대,2,09 등급없음,하,정보 제공 미동의,정보 제공 미동의,41,1,3.00,...,0.00,0.00,122.42,0.00,0.00,0.00,2.00,100.00,0.00,before
13,006ad32069efb2a8ce1a99542dfe41cb14e8e75685278a...,30대,1,09 등급없음,하,등급없음,등급없음,28,3,5.00,...,0.00,-63.72,250.34,0.00,0.00,0.00,3.00,81.10,18.90,after
15,007468e4eb9988503ce87ba186d8444a89a1c63aed6c76...,70대 이상,1,04 그린,중,01 위험회피형,02 안정추구형,44,3,49.00,...,0.00,-8.23,86.76,-0.04,0.00,-0.77,7.00,6.03,93.97,before
16,007dce6cfd89ac52c4ad7b6cfae34c8e59a77e476aeb53...,30대,1,09 등급없음,하,03 위험감수형,04 적극투자형,41,3,88.00,...,0.00,1.64,51.15,-0.72,-3.19,5.65,9.00,174.58,-74.58,before
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,f35baa477f23d0a24fbbd0b81c482fb99f0d98ad1f40a6...,60대,1,03 로얄,상,03 위험감수형,05 공격투자형,11,2,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,before
9996,f5f210f650a6b514b44cef5080ad5937bc4d978d2496bb...,30대,1,09 등급없음,하,02 위험중립형,03 위험중립형,41,1,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,before
9997,f80ac57c18e08540ee487d81ef71259e466e0e3501cd0e...,20대,1,09 등급없음,하,03 위험감수형,04 적극투자형,30,1,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,after
9998,f9e702b097617b92fe886608b1e4e2d1a3c53ee7b0b8f1...,60대,2,02 골드,상,03 위험감수형,05 공격투자형,11,10,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,after


In [142]:
df2 = cus_inf.loc[(cus_inf['age_cat']=='20대') & (cus_inf['sex'] == 1)].groupby(by=['covid']).median().reindex(['before','after']).reset_index()
df3 = cus_inf.loc[(cus_inf['age_cat']=='20대') & (cus_inf['sex'] == 2)].groupby(by=['covid']).median().reindex(['before','after']).reset_index()
df4 = cus_inf.loc[(cus_inf['age_cat']=='30대') & (cus_inf['sex'] == 1)].groupby(by=['covid']).median().reindex(['before','after']).reset_index()
df5 = cus_inf.loc[(cus_inf['age_cat']=='30대') & (cus_inf['sex'] == 2)].groupby(by=['covid']).median().reindex(['before','after']).reset_index()

trace3 = go.Bar(x=['코로나 이전','코로나 이후'], y=df2.freq, name='20대 남성',marker_color='#2C66B5')
trace4 = go.Bar(x=['코로나 이전','코로나 이후'], y=df3.freq, name='20대 여성',marker_color='#F3BE50')
trace5 = go.Bar(x=['코로나 이전','코로나 이후'], y=df4.freq, name='30대 남성',marker_color='#2C66B5')
trace6 = go.Bar(x=['코로나 이전','코로나 이후'], y=df5.freq, name='30대 여성',marker_color='#F3BE50')

fig = go.Figure(data=[trace3, trace4, trace5, trace6], layout=go.Layout(title='고객별 거래 빈도'))
fig.update_layout(font=dict(size=18,))
pyo.iplot(fig)